Note: Most code is copied from: https://www.kaggle.com/code/terencicp/steam-games-data-transformation

In [ ]:
import pandas as pd
import json
import numpy as np
from datetime import datetime

Load json version of games dataset

In [ ]:
first_file = '../data/games.json'
with open(first_file, 'r', encoding="utf8") as file:
    json_data = json.load(file)

Drop unneeded variables

In [ ]:
dropped = [
    'packages', 'screenshots', 'movies', 'score_rank', 'header_image',
    'reviews', 'website', 'support_url', 'notes', 'support_email',
    'median_playtime_2weeks', 'required_age',
    'metacritic_url', 'detailed_description', 'about_the_game', 
    'average_playtime_2weeks'
]

In [ ]:
# Process each game's information and store in a list
games = [{
    **{k: v for k, v in game_info.items() if k not in dropped},
    'tags': list(tags.keys()) if isinstance((tags := game_info.get('tags', {})), dict) else [],
    'tag_frequencies': list(tags.values()) if isinstance(tags, dict) else [],
    'app_id': app_id
} for app_id, game_info in json_data.items()]

# Create a DataFrame from the processed list
df = pd.DataFrame(games)
df

## Data Cleaning

Remove games with no sales:

In [ ]:
count = (df['estimated_owners'] == "0 - 0").sum()
print("Number of games with estimated owners '0 - 0':", count)

In [ ]:
df[df['estimated_owners'] == "0 - 0"]

Some games just seem to be developer tests. Let's remove them. We'll also remove games with no reviews or no categories:

In [ ]:
# Filter games without sales, reviews or categories
df2 = df[~((df['estimated_owners'] == "0 - 0") | (df['positive'] + df['negative'] == 0) | (df['categories'].str.len() == 0))]

Let's see the DataFrame again. It seems we got rid of more than 20000 irrelevant games:

In [ ]:
df2.shape

I'll also split the 'estimated_owners' column into two different variables, this way we'll be able to use it for aggregation in Tableau:

In [ ]:
# Split estimated_owners into two: min_owners and max_owners
df2[['min_owners', 'max_owners']] = df2['estimated_owners'].str.split(' - ', expand=True)

# Remove the original field
df2 = df2.drop('estimated_owners', axis=1)

In [ ]:
df2[['min_owners', 'max_owners']]

Let's have a look at the distribution of prices:

In [ ]:
# Box plot of price
df2.boxplot(column=['price'])

In [ ]:
# Games priced above 200$
df2[df2['price'] > 200]

We can see that the game priced at $999 is basically a cash-grab without any actual sales, and being an extreme outlier it can distort our analysis. Let's remove it:

In [ ]:
# Delete game with id 1200520
df2 = df2[df2['app_id'] != '1200520']

In [ ]:
# Games with median > 60 000 would mean that over half the playerbase has more than 1000 hours played,
# which is not very realistic (unless the game has very little players), so reset them to 0 in order to prevent skewing the data
# The entries which have median_playtime_forever > 60 000 are usually not really games, but overlays or software that runs in the background
df2.loc[df2.median_playtime_forever > 60000, 'median_playtime_forever'] = 0

## Normalizing data

The DataFrame contains fields such as 'categories' or 'tags' that consist on lists of values. To improve the performance of the visualization we'll build in Tableau we must convert this fields into separate tables, that will be linked with the main table using the 'app_id' column.

In [ ]:
# Create a separate DataFrame for each list-type column
cols = ["supported_languages", "full_audio_languages", "categories", "genres"]
for col in cols:
    new_df = df2.explode(col)[['app_id', col]]
    new_df.to_csv(f'../data/{col}.csv', index=False)

df_tags = df2.explode('tags')[['app_id', 'tags']]
df_frequencies = df2.explode('tag_frequencies')['tag_frequencies']
df_tags['tag_frequencies'] = df_frequencies.values

# Remove the list columns from the main DataFrame
columns_to_remove = cols + ['tags', 'tag_frequencies']
df_imploded = df2.drop(columns=columns_to_remove)

In [ ]:
df_imploded

In [ ]:
# Creating a custom score column, where each game is rated based on its 
# price, metacritic score, positive reviews, negative reviews and median playtime forever
# The values are first normalized so that they are all in the same range, 
# then they are multiplied by a weight and summed to get the final score

# Normalize the columns

# this term will provide a value in [0, 1], favoring lower prices (e.g. 1/sqrt(1) = 1, 1/sqrt(100) = 0.1)
# the max operation prevents division by 0 & makes it 
# so extremely low price doesn't affect the score that much (e.g. 1/sqrt(0.01) = 10)
price_normalized = 1/np.sqrt(np.maximum(df_imploded['price'], 1))

metacritic_score_normalized = df_imploded['metacritic_score'] / df_imploded['metacritic_score'].max()
positive_ratio = df_imploded['positive'] / np.maximum((df_imploded['positive']+df_imploded['negative']), 1) # ratio is in [0, 1]

# total reviews is a good indicator of how many people bought the game,
# so this term effectively serves as an "amount of sales" score
# the purpose of this is more so to filter out games with very few reviews 
# (where all the users left positive reviews or negative reviews)
total_reviews_normalized = (df_imploded['positive'] + df_imploded['negative']) / (df_imploded['positive'] + df_imploded['negative']).max()

# use median to prevent outliers from influencing
median_playtime_forever_normalized = df_imploded['median_playtime_forever'] / df_imploded['median_playtime_forever'].max()

# Assign weights based on intuition
price_weight = 0
metacritic_score_weight = 0 # Metacritic score is less important as not all games have it
positive_ratio_weight = 1.5 # Positive ratio is a good indicator of a game's quality
total_reviews_weight = 2.5 # High selling games have more reviews
median_playtime_forever_weight = 0 # Median playtime is easily manipulated in lower selling games

# Calculate score
df_imploded['score'] = (price_normalized * price_weight +
               metacritic_score_normalized * metacritic_score_weight +
               positive_ratio * positive_ratio_weight +
               total_reviews_normalized * total_reviews_weight +
               median_playtime_forever_normalized * median_playtime_forever_weight)

In [ ]:
# CS:GO comes out on top, which seems reasonable considering the popularity of the game
df_imploded[df_imploded['score'] == df_imploded['score'].max()]


## Save results as CSV

Dataset that doesn't include list-like columns

In [ ]:
df_imploded.to_csv('../data/cleaned_games.csv', index=False)